## Lenet 5 con MNIST

Imports

In [1]:
import tempfile
import os
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses

Load dataset

In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape

11490434/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

Normalize the input image so that each pixel value is between 0 to 1.

In [3]:
train_images = tf.pad(train_images, [[0, 0], [2,2], [2,2]])/255
test_images = tf.pad(test_images, [[0, 0], [2,2], [2,2]])/255
train_images.shape

TensorShape([60000, 32, 32])

Most CNN’s accept 4-dimensional tensors as inputs having the dimensions of batch size, height, width, and channel. Since MNIST images are grayscale, the last dimension does not necessarily exist. We need to expand the tensor and create a dummy dimension at axis number 3. (Recall that the tensor initially had axis 0, 1, and 2.)

In [4]:
train_images = tf.expand_dims(train_images, axis=3, name=None)
test_images = tf.expand_dims(test_images, axis=3, name=None)
train_images.shape

TensorShape([60000, 32, 32, 1])

The last 2000 samples of the training set are reserved for the validation set.

In [5]:
val_images = train_images[-2000:,:,:,:]
val_labels = train_labels[-2000:]
train_images = train_images[:-2000,:,:,:]
train_labels = train_labels[:-2000]

Define the model architecture.

In [6]:
model = models.Sequential()
model.add(layers.SeparableConv2D(6, 5, activation='tanh', input_shape=train_images.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.SeparableConv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.SeparableConv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separable  (None, 28, 28, 6)        37        
 Conv2D)                                                         
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 separable_conv2d_1 (Separab  (None, 10, 10, 16)       262       
 leConv2D)                                                       
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                            

Train model

In [7]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=64, epochs=40, validation_data=(val_images, val_labels))

Epoch 1/40
907/907 [==============================] - 61s 63ms/step - loss: 1.1946 - accuracy: 0.5717 - val_loss: 0.3601 - val_accuracy: 0.8915
Epoch 2/40
907/907 [==============================] - 64s 70ms/step - loss: 0.4086 - accuracy: 0.8722 - val_loss: 0.2644 - val_accuracy: 0.9200
Epoch 3/40
907/907 [==============================] - 56s 62ms/step - loss: 0.3115 - accuracy: 0.9031 - val_loss: 0.1841 - val_accuracy: 0.9420
Epoch 4/40
907/907 [==============================] - 56s 62ms/step - loss: 0.2643 - accuracy: 0.9163 - val_loss: 0.1570 - val_accuracy: 0.9580
Epoch 5/40
907/907 [==============================] - 57s 63ms/step - loss: 0.2296 - accuracy: 0.9284 - val_loss: 0.1354 - val_accuracy: 0.9630
Epoch 6/40
907/907 [==============================] - 56s 62ms/step - loss: 0.2064 - accuracy: 0.9361 - val_loss: 0.1263 - val_accuracy: 0.9660
Epoch 7/40
907/907 [==============================] - 56s 62ms/step - loss: 0.1886 - accuracy: 0.9410 - val_loss: 0.1239 - val_accuracy:

Evaluate

In [8]:
start = time.time()
accuracy = model.evaluate(test_images, test_labels)
end = time.time()
print("Accuracy :",accuracy[1])
print("Images per second :", int(test_images.shape[0]/(end - start)))

313/313 [==============================] - 5s 14ms/step - loss: 0.0800 - accuracy: 0.9756
Accuracy : 0.975600004196167
Images per second : 1926


In [9]:
# Guardar el Modelo
model.save('model_SepConv2.h5')
